In [ ]:
#Import required packages
from torchdiffeq import odeint_adjoint as odeint
import numpy as np 
import torch.nn as nn
import torch.optim as optim
import torch

In [ ]:
#Instantiate functions to compute activation and repression equations
def activation(x, k, theta, n):
    return (k*(x/theta)**n)/(1+(x/theta)**n)

def repression(x, k, theta, n):
    return k/(1+(x/theta)**n)

def nonlinearity(x, kc, km):
    return (kc*x)/(km+x)

In [ ]:
#Define custom Module with fixed dual control architecture
class DualControl(nn.Module):
    def __init__(self):
        '''Constructor instantiating weights as model parameters and constants'''
        super().__init__()
        
        #Initialize constants, taken from Verma et al paper.
        self.Vin = 1.
        self.e0 = 0.0467
        self.lam = 1.93E-4 #1/s
        #Assume equal kinetics for all three enzymes
        self.kc = 12
        self.km = 10 #1/s

        #Initizalize weights for training
        self.W = nn.Parameter(torch.from_numpy(np.array([[2,2],[1,1], [1E-7, 1E-7]]))) 
        #parameters are n1, n2, theta1, theta2, k1, k2
        
    def forward(self, t, y):
        '''Computes derivatives of system of differential equations'''
        ydot = torch.zeros(6)
        ydot[0] = self.Vin - self.lam*y[0] - self.e0*nonlinearity(y[0], self.kc, self.km) - self.lam*y[1]
        ydot[1] = y[2]*nonlinearity(y[0], self.kc, self.km) - y[3]*nonlinearity(y[1], self.kc, self.km) - self.lam*y[1]
        ydot[2] = repression(y[1], self.W[2][0], self.W[1][0], self.W[0][0]) - self.lam*y[2]
        ydot[3] = activation(y[1], self.W[2][1], self.W[1][1], self.W[0][1]) - self.lam*y[3]
        ydot[4] = (self.Vin -  y[3]*nonlinearity(y[1], self.kc, self.km))**2
        ydot[5] = repression(y[1], self.W[2][0], self.W[1][0], self.W[0][0]) + activation(y[1], self.W[2][1], self.W[1][1], self.W[0][1])
        return ydot

In [ ]:
#Custom loss function
def my_loss(solution, alpha=1):
    """Computes scalarized loss including genetic constraint and product production"""
    j1 = solution[-1][-2]
    j2 = solution[-1][-1]
    loss = j1 + alpha*j2
    return loss

In [ ]:
#Establish initial conditions
y0 = torch.from_numpy(np.array([2290., 0., 0., 0., 0., 0.]))
t = torch.from_numpy(np.linspace(1,5e4,200))

#Construct model by instantiating class defined above
model = DualControl()

#Set hyperparameters
learning_rate = 0.01
weight_decay_coefficient = 0
num_epochs = 10
criterion = my_loss

#Establish optimizer - settings from torchdiffeq package examples
optimizer = optim.Adam(model.parameters(), amsgrad=False,
                                    weight_decay=weight_decay_coefficient, lr=learning_rate)

model.train()  #Puts model in training mode
              
for i in range(num_epochs):
    # Zero gradients
    optimizer.zero_grad()

    # Forward pass: Compute steady-state solution
    print(model.parameters())
    y_pred = odeint(model, y0, t, method='dopri5')
    

    # Compute and print loss
    loss = criterion(y_pred)

    print('Epoch',  i, 'Loss', loss.item())

    # Perform a backward pass, and update the weights.
    loss.backward()
    optimizer.step()

In [ ]:
#Oisin's example rewritten with my functions
import matplotlib.pyplot as plt
import numpy as np
import torch 
#from scipy.integrate import odeint
from torch.autograd import grad

from torchdiffeq import odeint_adjoint
from torchdiffeq import odeint

np.random.seed(2021)
torch.manual_seed(2021)
plt.close('all')


def loss_fun(pred, alpha=1):
    """Computes scalarized loss including genetic constraint and product production"""
    j1 = pred[-1][-2]
    j2 = pred[-1][-1]
    loss = j1 + alpha*j2
    return loss

class ODEFun(torch.nn.Module):
    def __init__(self):
        super(ODEFun, self).__init__()
        #Initialize constants, taken from Verma et al paper.
        self.Vin = 1.
        self.e0 = 0.0467
        self.lam = 1.93E-4 #1/s
        #Assume equal kinetics for all three enzymes
        self.kc = 12
        self.km = 10 #1/s

        self.W = torch.nn.Parameter(torch.tensor([[2,2],[1,1], [1E-7, 1E-7]]), requires_grad=True)
        

    def forward(self, t, y):
        '''Computes derivatives of system of differential equations'''
        dx0 = self.Vin - self.lam*y[0] - self.e0*nonlinearity(y[0], self.kc, self.km) - self.lam*y[1]
        dx1 = y[2]*nonlinearity(y[0], self.kc, self.km) - y[3]*nonlinearity(y[1], self.kc, self.km) - self.lam*y[1]
        de1 = repression(y[1], self.W[2][0], self.W[1][0], self.W[0][0]) - self.lam*y[2]
        de2 = activation(y[1], self.W[2][1], self.W[1][1], self.W[0][1]) - self.lam*y[3]
        j1 = (self.Vin -  y[3]*nonlinearity(y[1], self.kc, self.km))**2
        j2 = repression(y[1], self.W[2][0], self.W[1][0], self.W[0][0]) + activation(y[1], self.W[2][1], self.W[1][1], self.W[0][1])
        return torch.stack([dx0, dx1, de1, de2, j1, j2])
        
t = torch.linspace(0, 5E-4, 100) 
true_y0 = torch.tensor([2290., 0., 0., 0., 0., 0.])

func = ODEFun()
lr = 0.0000001
num_iters = 100
optimizer = optim.Adam(model.parameters(), amsgrad=False,
                                    weight_decay=weight_decay_coefficient, lr=learning_rate)

print('itr loss theta_0 theta_1')
for ii in range(num_iters): 
    
    optimizer.zero_grad()
    
    pred = odeint(func, true_y0, t, method='dopri5')
    #pred = odeint_adjoint(func, true_y0, t, method='dopri5')
    loss = loss_fun(pred)
    
    loss.backward()
    optimizer.step()
    
    if ii % 10 == 0:
        print(ii, round(loss.item(), 4), round(func.W[0][0].item(), 4), round(func.W[1][0].item(), 4))
    

print('\n')
print('Pred: theta_0: {:2.2}, theta_1: {:2.2}'.format(round(func.W[0][0].item(), 4), round(func.W[1][0].item(), 4)))


In [ ]:
#Oisin's example
import matplotlib.pyplot as plt
import numpy as np
import torch 
#from scipy.integrate import odeint
from torch.autograd import grad

from torchdiffeq import odeint_adjoint
from torchdiffeq import odeint

np.random.seed(2021)
torch.manual_seed(2021)
plt.close('all')



def loss_fun(gt, pred):
    return ((gt-pred)**2).mean()
    

class ODEFun(torch.nn.Module):
    def __init__(self):
        super(ODEFun, self).__init__()
        self.theta = torch.nn.Parameter(torch.tensor([0.0, 0.0]), requires_grad=True)
        
    def forward(self, t, y):
        S, I = y
        ds = -self.theta[0]*S*I
        di = self.theta[1]*S*I - I
        return torch.stack([ds, di])

def gt_fun(t, y):
    theta_0 = 5.5
    theta_1 = 8.0
    S, I = y
    ds = -theta_0*S*I
    di = theta_1*S*I - I
    return torch.stack([ds, di])
        
true_y0 = torch.tensor([0.99, 0.01])
t = torch.linspace(0, 5, 100) 


# gt
with torch.no_grad():
    true_y = odeint(gt_fun, true_y0, t, method='dopri5')
    true_y_noise = true_y + torch.randn(true_y.shape)*0.02
    #true_y_adj = odeint_adjoint(gt_fun, true_y0, t, method='dopri5')


func = ODEFun()
lr = 0.01
num_iters = 1000
optimizer = torch.optim.RMSprop(func.parameters(), lr=lr)

print('itr loss theta_0 theta_1')
for ii in range(num_iters): 
    
    optimizer.zero_grad()
    
    pred = odeint(func, true_y0, t, method='dopri5')
    #pred = odeint_adjoint(func, true_y0, t, method='dopri5')
    loss = loss_fun(true_y_noise, pred)
    
    loss.backward()
    optimizer.step()
    
    if ii % 10 == 0:
        print(ii, round(loss.item(), 4), round(func.theta[0].item(), 4), round(func.theta[1].item(), 4))
    

# Note: here the GT values are hardcoded - fix
print('\n')
print('GT  : theta_0: {:2.2}, theta_1: {:2.2}'.format(5.5, 8.0))
print('Pred: theta_0: {:2.2}, theta_1: {:2.2}'.format(round(func.theta[0].item(), 4), round(func.theta[1].item(), 4)))



plt.figure(1)
with torch.no_grad():
    plt.plot(t, true_y[:,0], 'C0', lw=3, label='S GT')
    plt.plot(t, true_y[:,1], 'C1', lw=3, label='I GT')
    plt.plot(t, pred[:,0], 'k', lw=1, label='S Pred')
    plt.plot(t, pred[:,1], ':k', lw=1, label='I Pred')
plt.legend()
plt.xlabel('time')
plt.show()